In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

In [ ]:
import pandas as pd

data = pd.read_csv('/content/drive/My Drive/data_cs334/rewritten_texts_csv.csv', encoding='ISO-8859-1')

file_path_score = '/content/drive/My Drive/data_cs334/Gemma_Rewriting_Score_Descriptors.txt'

with open(file_path_score, 'r') as file:
    score = file.read()


In [ ]:
def construct_input(row, scores):
    prompt = scores + '\n\n'
    #prompt += 'Here are a few grading examples: \n\n'
    #prompt += examples + '\n\n'
    prompt += 'Original text:\n\n'
    prompt += row['original_text'] + '\n\n'
    prompt += 'Prompt to rewrite the original text:\n\n'
    prompt += row['prompt'] + '\n\n'
    prompt += 'Rewritten text:\n\n'
    prompt += row['rewritten_text'] + '\n\n'
    prompt += 'Score: \n\n'
    return prompt


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import csv

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
tokenizer.model_max_length = 4096

with open('/content/drive/My Drive/data_cs334/Gemma_Rewriting_Score_Descriptors.txt', 'r') as file:
    score_descriptors = file.read()

start = 0
end = 10000


with open('/content/drive/My Drive/data_cs334/evaluation_results.csv', mode='a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    #writer.writerow(["Prompt", "Original Text", "Rewritten Text", "Score"])

    for index in range(start, end):
        row = data.iloc[index]
        input_text = construct_input(row, score_descriptors)
        inputs = tokenizer(input_text, return_tensors="pt")
        outputs = model.generate(**inputs, max_new_tokens=50)
        score = tokenizer.batch_decode(outputs, skip_special_tokens=True)

        writer.writerow([row['prompt'], row['original_text'], row['rewritten_text'], score])
        print('Completed index: ' + str(index) + '\n')


